In [1]:
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.io as pio
import ta
import pandas as pd
import numpy as np
import yfinance as yf
from pandas import DataFrame
# Set the default template to plotly_dark
pio.templates.default = 'plotly_dark'

In [7]:
import yfinance as yf
import pandas as pd


class StockData:
    """
    A class that retrieves historical stock data from Yahoo Finance using the yfinance library.

    Attributes:
        symbol (str): The stock symbol to retrieve data for.

        start_date (str): The start date of the date range to retrieve data for, in YYYY-MM-DD format.

        end_date (str): The end date of the date range to retrieve data for, in YYYY-MM-DD format.

        interval (str): The time interval between data points. Valid values are '1d' (daily), '1wk' (weekly), '1mo' (monthly), '1m' (1-minute), '2m' (2-minute), '5m' (5-minute), '15m' (15-minute), '30m' (30-minute), '60m' (60-minute), '90m' (90-minute), and '1h' (hourly).

        data (pandas.DataFrame): The retrieved stock data.
    """

    def __init__(self, symbol, start_date=None, end_date=None, interval="1d") -> None:

        self.startup(symbol, start_date, end_date, interval)

    def startup(self, symbol, start_date, end_date, interval) -> None:
        """
        Constructs a new StockData object and downloads the stock data from Yahoo Finance.

        Args:
            symbol (str): The stock symbol to retrieve data for.

            start_date (str): The start date of the date range to retrieve data for, in YYYY-MM-DD format.

            end_date (str): The end date of the date range to retrieve data for, in YYYY-MM-DD format.

            interval (str, optional): The time interval between data points. Default is '1d' for daily data. Valid values are '1d' (daily), '1wk' (weekly), '1mo' (monthly), '1m' (1-minute), '2m' (2-minute), '5m' (5-minute), '15m' (15-minute), '30m' (30-minute), '60m' (60-minute), '90m' (90-minute), and '1h' (hourly).
        """
        self.symbol: str = symbol
        self.start_date: str = start_date
        self.end_date: str = end_date
        self.interval: str = interval
        self.data: DataFrame = yf.download(
            symbol, start=start_date, end=end_date, interval=interval
        )

    def __repr__(self) -> str:
        """
        Returns a string representation of the StockData object.
        """
        return f"StockData(symbol={self.symbol}, start_date={self.start_date}, end_date={self.end_date}, interval={self.interval})"

    def __str__(self) -> str:
        """
        Returns a string representation of the stock data.
        """
        return str(self.data)

In [10]:
stockdata = StockData(
    symbol='TSLA'
)
df: DataFrame = stockdata.data

[*********************100%***********************]  1 of 1 completed
